In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
Data=pd.read_csv('../input/digit-recognizer/train.csv')

In [ ]:
Data.head()

In [ ]:
Y=np.array(Data['label'])
X=np.array(Data.drop('label',axis=1))/ 255.

In [ ]:
X=X.reshape((-1,28,28,1))

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(X[25].reshape(28,28))
print(Y[25])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,BatchNormalization,Dropout,Conv2D,MaxPool2D

In [ ]:
print(tf.config.list_physical_devices('GPU'),'//',tf.test.is_built_with_cuda())

In [ ]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=12,
    width_shift_range=0.12,
    height_shift_range=0.12,
    shear_range=0.12,
    validation_split=0.2
)

In [ ]:
training_generator = datagen.flow(X_train, y_train, batch_size=32,subset='training')
validation_generator = datagen.flow(X_train, y_train, batch_size=32,subset='validation')

In [ ]:
plt.figure(figsize=(10,5))
for i in range(6):
    plt.subplot(2,3,i+1)
    for x,y in training_generator:
        plt.imshow((x[0]/255).reshape(28,28),cmap='gray')
        plt.title('y={}'.format(y[0]))
        plt.axis('off')
        break
plt.tight_layout()
plt.show()

In [ ]:
model=Sequential()

In [ ]:
model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same',activation ='relu', input_shape = (28,28,1)))
model.add(BatchNormalization())
model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.3))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(256, activation = "relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

In [ ]:
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=tf.keras.optimizers.RMSprop(lr=0.003, rho=0.9, epsilon=1e-08, decay=0.0),
    metrics=["accuracy"]
)

In [ ]:
cb=tf.keras.callbacks.EarlyStopping(patience=10,restore_best_weights=True)
learning_rate_reduction = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [ ]:
model.fit(training_generator,epochs=100,callbacks=[cb,learning_rate_reduction],validation_data=validation_generator)

In [ ]:
his=model.history.history

In [ ]:
plt.plot(his['accuracy'])
plt.plot(his['val_accuracy'])

In [ ]:
plt.plot(his['loss'])
plt.plot(his['val_loss'])

In [ ]:
model.evaluate( X_test, y_test)

**PREDICTIONS**

In [ ]:
pred_Data=np.array(pd.read_csv('../input/digit-recognizer/test.csv')/ 255.)
X_pred=pred_Data.reshape((-1,28,28,1))

In [ ]:
predictions=model.predict_classes(X_pred)

In [ ]:
submit=pd.DataFrame({'ImageId':range(1,len(predictions)+1),'Label':predictions})
submit.to_csv('submission.csv',index=False)